<a href="https://colab.research.google.com/github/kavya-avula/Fake_news_classification/blob/main/combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langdetect joblib scikit-learn --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# 🧠 STEP 2: Import Libraries
# =============================================
import re
import joblib
from langdetect import detect
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [ ]:
# 🌍 STEP 3: Load Models
# =============================================

# --- English TF-IDF + Logistic Regression ---
english_vectorizer = joblib.load("english_vectorizer.pkl")
english_model = joblib.load("english_model.pkl")

# --- Hindi TF-IDF + Logistic Regression ---
hindi_vectorizer = joblib.load("hindi_vectorizer.pkl")
hindi_model = joblib.load("hindi_model.pkl")

# --- Hindi SBERT + Logistic Regression (Optional Transformer) ---
with open("hindi_sbert_model_name.txt", "r") as f:
    hindi_sbert_model_name = f.read().strip()

hindi_sbert_tokenizer = AutoTokenizer.from_pretrained(hindi_sbert_model_name)
hindi_sbert_model = AutoModelForSequenceClassification.from_pretrained(hindi_sbert_model_name)
hindi_sbert_lr = joblib.load("hindi_sbert_model.pkl")

print("✅ All models loaded successfully!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ All models loaded successfully!


In [ ]:
# 🧹 STEP 4: Text Cleaning
# =============================================
def clean_text(text):
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^A-Za-z\u0900-\u097F\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
# 🌐 STEP 5: Language Detection
# =============================================
def detect_language(text):
    try:
        lang = detect(text)
        if lang == "hi":
            return "hindi"
        elif lang == "en":
            return "english"
        else:
            return "unknown"
    except:
        return "unknown"

In [ ]:
# 🧠 STEP 6: Unified Fake News Prediction
# =============================================
def predict_news(text):
    text = clean_text(text)
    lang = detect_language(text)

    if lang == "english":
        vector = english_vectorizer.transform([text])
        prediction = english_model.predict(vector)[0]
        return "📰 English Detected → " + ("🧢 FAKE NEWS" if prediction == 1 else "✅ REAL NEWS")

    elif lang == "hindi":
        # Option 1: Use Hindi TF-IDF model
        vector = hindi_vectorizer.transform([text])
        pred_tfidf = hindi_model.predict(vector)[0]

In [ ]:
 # 🧠 STEP 6: Unified Fake News Prediction
# =============================================
def predict_news(text):
    text = clean_text(text)
    lang = detect_language(text)

    if lang == "english":
        # ---- English Model ----
        vector = english_vectorizer.transform([text])
        prediction = english_model.predict(vector)[0]
        return "📰 English Detected → " + ("🧢 FAKE NEWS" if prediction == 1 else "✅ REAL NEWS")

    elif lang == "hindi":
        # ---- Hindi TF-IDF Model ----
        vector = hindi_vectorizer.transform([text])
        pred_tfidf = hindi_model.predict(vector)[0]

        # ---- Hindi SBERT + Logistic Regression ----
        inputs = hindi_sbert_tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=128
        )
        with torch.no_grad():
            # Extract [CLS] embedding from SBERT
            embeddings = hindi_sbert_model.base_model(**inputs).last_hidden_state[:, 0, :].cpu().numpy()
        pred_sbert = hindi_sbert_lr.predict(embeddings)[0]

        # ---- Combine results ----
        if pred_tfidf == pred_sbert:
            final = pred_tfidf
        else:
            # fallback to SBERT if disagreement
            final = pred_sbert

        return "🗞️ Hindi Detected → " + ("🧢 FAKE NEWS" if final == 1 else "✅ REAL NEWS")

    else:
        return "⚠️ Language not recognized (only supports Hindi & English)."


In [ ]:
from langdetect import detect

def predict_news(text):
    lang = detect(text)
    text_clean = clean_text(text)
    text_proc = preprocess(text_clean) if lang == "hi" else text_clean.lower()
    vector = tfidf.transform([text_proc])
    prediction = model.predict(vector)[0]
    lang_label = "Hindi" if lang == "hi" else "English"
    result = "🧢 FAKE NEWS" if prediction == 1 else "✅ REAL NEWS"
    print(f"🗞️ {lang_label} Detected → {result}")


In [ ]:
def predict_news(text):
    text = clean_text(text)
    lang = detect_language(text)

    if lang == "english":
        # ---- English Model ----
        vector = english_vectorizer.transform([text])
        prediction = english_model.predict(vector)[0]
        return "📰 English Detected → " + ("🧢 FAKE NEWS" if prediction == 1 else "✅ REAL NEWS")

    elif lang == "hindi":
        # ---- Hindi TF-IDF Model ----
        vector = hindi_vectorizer.transform([text])
        pred_tfidf = hindi_model.predict(vector)[0]

        # ---- Hindi SBERT + Logistic Regression ----
        inputs = hindi_sbert_tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=128
        )

        with torch.no_grad():
            embeddings = hindi_sbert_model.base_model(**inputs).last_hidden_state[:, 0, :].cpu().numpy()
            pred_sbert = hindi_sbert_lr.predict(embeddings)[0]

        # ---- Combine both predictions ----
        if pred_tfidf == pred_sbert:
            final = pred_tfidf
        else:
            final = pred_sbert  # fallback to SBERT if disagreement

        return "🗞️ Hindi Detected → " + ("🧢 FAKE NEWS" if final == 1 else "✅ REAL NEWS")

    else:
        return "⚠️ Language not recognized (only supports Hindi & English)."


In [ ]:
samples = [
    "Breaking news: Scientists discovered a miracle cure for cold!",
    "प्रधानमंत्री ने कहा कि देश की अर्थव्यवस्था तेजी से बढ़ रही है।",
    "यह खबर पूरी तरह झूठी है और किसी स्रोत से प्रमाणित नहीं है।"
]

for text in samples:
    print("\nInput:", text)
    print("Result:", predict_news(text))



Input: Breaking news: Scientists discovered a miracle cure for cold!
Result: 📰 English Detected → 🧢 FAKE NEWS

Input: प्रधानमंत्री ने कहा कि देश की अर्थव्यवस्था तेजी से बढ़ रही है।
Result: 🗞️ Hindi Detected → ✅ REAL NEWS

Input: यह खबर पूरी तरह झूठी है और किसी स्रोत से प्रमाणित नहीं है।
Result: 🗞️ Hindi Detected → ✅ REAL NEWS
